In [40]:
import sqlalchemy
import pandas as pd
from sqlalchemy import text

In [41]:
# load tables
df_brands = pd.read_csv('data/df_brands.csv', index_col=0)
df_items = pd.read_csv('data/df_items.csv', index_col=0)
df_receipts = pd.read_csv('data/df_receipts.csv', index_col=0)
df_reward_receipt_items = pd.read_csv('data/df_reward_receipt_items.csv', index_col=0)
df_users = pd.read_csv('data/df_users.csv', index_col=0)

# create db
engine = sqlalchemy.create_engine('sqlite:///fetch_rewards.db', echo = True)
df_brands.set_index('id', inplace=True)
df_brands.to_sql('Brands', con=engine, if_exists='replace', index=True)
df_items.to_sql('Items', con=engine, if_exists='replace', index_label='id')
df_receipts.set_index('id', inplace=True)
df_receipts.to_sql('Receipts', con=engine, if_exists='replace', index=True)
df_reward_receipt_items.to_sql('RewardsReceiptsItems', con=engine, if_exists='replace', index_label='id')
df_users.set_index('id', inplace=True)
df_users.to_sql('Users', con=engine, if_exists='replace', index=True)

2025-01-22 23:37:08,624 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Brands")
2025-01-22 23:37:08,625 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-22 23:37:08,628 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Brands")
2025-01-22 23:37:08,629 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-22 23:37:08,632 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2025-01-22 23:37:08,633 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-22 23:37:08,635 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Brands")
2025-01-22 23:37:08,636 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-22 23:37:08,638 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2025-01-22 23:37:08,639 INFO sqlalchemy.engine.Engine [raw sql] ('Brands',)
2025-01-22 23:37:08,640 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("Brands")
202

495

### What are the top 5 brands by receipts scanned for most recent month?

In [44]:
# get most recent date
most_recent_date = df_receipts['dateScanned'].max()
most_recent_date

'2021-03-01 15:17:34.772'

In [58]:
query = text(
f'''
SELECT t1.brandId, t4.name, COUNT(DISTINCT t2.receiptsId) AS cnt
FROM Items t1
JOIN RewardsReceiptsItems t2 ON t1.id = t2.itemId
JOIN Receipts t3 ON t2.receiptsId = t3.id
JOIN Brands t4 ON t1.brandId = t4.id
WHERE t3.dateScanned >= DATE('{most_recent_date}', '-1 month')
GROUP BY t1.brandId, t4.name
ORDER BY 3 DESC;
'''
)
with engine.connect() as connection:
    result = connection.execute(query)
    print("Result: ")
    for row in result:
        print(row)

2025-01-23 00:27:39,296 INFO sqlalchemy.engine.Engine 
SELECT t1.brandId, t4.name, COUNT(DISTINCT t2.receiptsId) AS cnt
FROM Items t1
JOIN RewardsReceiptsItems t2 ON t1.id = t2.itemId
JOIN Receipts t3 ON t2.receiptsId = t3.id
JOIN Brands t4 ON t1.brandId = t4.id
WHERE t3.dateScanned >= DATE('2021-03-01 15:17:34.772', '-1 month')
GROUP BY t1.brandId, t4.name
ORDER BY 3 DESC;

2025-01-23 00:27:39,299 INFO sqlalchemy.engine.Engine [cached since 1565s ago] ()
Result: 
('5bd201f090fa074576779a1a', 'Viva', 1)


There aren't many valid results due to missing values, but the query shows the approach taken to address the problem.

### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [59]:
query = text(
'''
SELECT 
    AVG(CASE WHEN rewardsReceiptStatus = 'REJECTED' THEN totalSpent END) AS AvgRejected,
    AVG(CASE WHEN rewardsReceiptStatus = 'FINISHED' THEN totalSpent END) AS AvgAccepted
FROM Receipts;
'''
)
with engine.connect() as connection:
    result = connection.execute(query)
    print("Result: ")
    for row in result:
        print(row)

2025-01-23 00:27:41,806 INFO sqlalchemy.engine.Engine 
SELECT 
    AVG(CASE WHEN rewardsReceiptStatus = 'REJECTED' THEN totalSpent END) AS AvgRejected,
    AVG(CASE WHEN rewardsReceiptStatus = 'FINISHED' THEN totalSpent END) AS AvgAccepted
FROM Receipts;

2025-01-23 00:27:41,807 INFO sqlalchemy.engine.Engine [cached since 2107s ago] ()
Result: 
(23.326056338028184, 80.85430501930502)


Accepted is greater.

### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [60]:
query = text(
'''
SELECT 
    SUM(CASE WHEN rewardsReceiptStatus = 'REJECTED' THEN purchasedItemCount END) AS SumRejectedCount,
    SUM(CASE WHEN rewardsReceiptStatus = 'FINISHED' THEN purchasedItemCount END) AS SumAcceptedCount
FROM Receipts;
'''
)
with engine.connect() as connection:
    result = connection.execute(query)
    print("Result: ")
    for row in result:
        print(row)

2025-01-23 00:27:43,503 INFO sqlalchemy.engine.Engine 
SELECT 
    SUM(CASE WHEN rewardsReceiptStatus = 'REJECTED' THEN purchasedItemCount END) AS SumRejectedCount,
    SUM(CASE WHEN rewardsReceiptStatus = 'FINISHED' THEN purchasedItemCount END) AS SumAcceptedCount
FROM Receipts;

2025-01-23 00:27:43,504 INFO sqlalchemy.engine.Engine [cached since 1956s ago] ()
Result: 
(173.0, 8184.0)


Accepted is greater.

### Which brand has the most spend among users who were created within the past 6 months?

In [61]:
query = text(
f'''
SELECT t1.brandId, t5.name, SUM(t2.finalPrice) AS spent
FROM Items t1
JOIN RewardsReceiptsItems t2 ON t1.id = t2.itemId
JOIN Receipts t3 ON t2.receiptsId = t3.id
JOIN Users t4 ON t4.id = t3.UserId
JOIN Brands t5 ON t1.brandId = t5.id
WHERE t4.createdDate >= DATE('{most_recent_date}', '-6 month')
GROUP BY t1.brandId
ORDER BY 3 DESC
LIMIT 1;
'''
)
with engine.connect() as connection:
    result = connection.execute(query)
    print("Result: ")
    for row in result:
        print(row)

2025-01-23 00:27:45,131 INFO sqlalchemy.engine.Engine 
SELECT t1.brandId, t5.name, SUM(t2.finalPrice) AS spent
FROM Items t1
JOIN RewardsReceiptsItems t2 ON t1.id = t2.itemId
JOIN Receipts t3 ON t2.receiptsId = t3.id
JOIN Users t4 ON t4.id = t3.UserId
JOIN Brands t5 ON t1.brandId = t5.id
WHERE t4.createdDate >= DATE('2021-03-01 15:17:34.772', '-6 month')
GROUP BY t1.brandId
ORDER BY 3 DESC
LIMIT 1;

2025-01-23 00:27:45,133 INFO sqlalchemy.engine.Engine [cached since 1651s ago] ()
Result: 
('592486bee410d61fcea3d130', 'KNORR', 733.0600000000004)
